# Day 22
https://adventofcode.com/2016/day/22

In [1]:
import aocd
data = aocd.get_data(year=2016, day=22)

In [2]:
from collections import Counter, deque
from dataclasses import dataclass
from itertools import permutations
from typing import Set
import re

##### Part 1: Find viable pairs of nodes

In [3]:
re_node = re.compile(r'node-x(\d+)-y(\d+)\s+(\d+)T\s+(\d+)T\s+(\d+)T\s+(\d+)%')

In [4]:
@dataclass(frozen=True)
class Point():
    x: int
    y: int
        
    def taxicab_distance_to(self, other):
        return abs(self.x - other.x) + abs(self.y - other.y)
    
    def __add__(self, other):
        return Point(self.x+other.x, self.y+other.y)

In [5]:
@dataclass(frozen=True)
class Node():
    location: Point
    size: int
    used: int
    avail: int
    
    def forms_viable_pair_with(self, other):
        return self.used > 0 and self.used <= other.avail
    
    @classmethod
    def from_match_groups(cls, groups):
        x, y, size, used, avail, percent = map(int, groups)
        return cls(Point(x, y), size, used, avail)
    
    @classmethod
    def all_from_df_text(cls, text):
        return [cls.from_match_groups(groups) for groups in re_node.findall(text)]

In [6]:
def all_viable_pairs(nodes):
    return [(a, b) for (a, b) in permutations(nodes, 2) if a.forms_viable_pair_with(b)]

In [7]:
nodes = Node.all_from_df_text(data)
p1 = len(all_viable_pairs(nodes))
print('Part 1: {}'.format(p1))

Part 1: 892


In [8]:
pairs = all_viable_pairs(nodes)
counted = Counter([a.location for a, b in pairs] + [b.location for a, b in pairs])
[(p, c) for (p, c) in counted.items() if c > 1]

[(Point(x=7, y=17), 892)]

##### Part 2: Solve the movement of the data

In [9]:
@dataclass(frozen=True)
class Situation():
    reachable: Set[Point]
    empty: Point
    target: Point
    moves: int
    
    @classmethod
    def from_node_list(cls, nodelist):
        pairs = all_viable_pairs(nodes)
        count = Counter([a.location for a, b in pairs] + [b.location for a, b in pairs])
        reachable = set(count.keys())
        empty = next(location for location, c in count.items() if c > 1)
        target = Point(max(loc.x for loc in reachable), 0)
        return cls(reachable, empty, target, 0)
    
    @property
    def is_valid_state(self):
        return self.empty in self.reachable and self.target in self.reachable
    
    @property
    def description(self):
        return 'empty:({},{}),target:({},{})'.format(self.empty.x, self.empty.y, self.target.x, self.target.y)
    
    def move_empty_cell(self, direction):
        new_empty = self.empty + direction
        return Situation(
            self.reachable,
            new_empty,
            self.empty if new_empty == self.target else self.target,
            self.moves+1
        )
    
    def all_possible_moves(self):
        return [move for move in [self.move_empty_cell(direction)
                                  for direction in (Point(0, -1), Point(0, 1), Point(-1, 0), Point(1, 0))]
                if move.is_valid_state]

In [10]:
def find_shortest_solution(nodes):
    initial = Situation.from_node_list(nodes)
    visited = set([initial.description])
    search = deque([initial])
    
    while search:
        candidate = search.popleft()
        if candidate.target == Point(0, 0):
            return candidate.moves
        for potential_next_move in candidate.all_possible_moves():
            if potential_next_move.description not in visited:
                visited.add(potential_next_move.description)
                search.append(potential_next_move)

In [11]:
p2 = find_shortest_solution(nodes)
print('Part 2: {}'.format(p2))

Part 2: 227
